## Install huggingface transformers

In [1]:
!pip install transformers
!pip install pytorch-nlp


fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import torch                                        # root package
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading


In [3]:
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::733425554560:role/service-role/AmazonSageMaker-ExecutionRole-20200504T094270


## 1. Obtain dataset

### We don't want to use the prepared dataset as it, wo we compare the sample dataset into csv

In [4]:
import csv
from torchnlp.datasets import imdb_dataset
train, test = imdb_dataset(train=True,test=True)

with open('data/train.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in train:
        csvwriter.writerow([i['text'],i['sentiment']])

with open('data/test.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['text','sentiment'])
    for i in test:
        csvwriter.writerow([i['text'],i['sentiment']])

aclImdb_v1.tar.gz: 84.1MB [00:12, 6.83MB/s]                              


In [40]:
import pandas as pd
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
mapping = {'neg': -1, 'pos': 1}
train_df = train_df.replace({'sentiment': mapping})
test_df = test_df.replace({'sentiment': mapping})

In [43]:
train_df.sample(10)

,text,sentiment
3069,"This film, was one of my childhood favorites a...",1
896,Gillian Holroyd (Kim Novak) is a witch. Secret...,1
17094,"When I first tuned in on this morning news, I ...",-1
2586,"I have NOT seen this movie, but I must. Having...",1
14280,They had an opportunity to make one of the bes...,-1
7235,You better see this episode from the beginning...,1
23309,"This movie is so awful, it is hard to find the...",-1
9145,After reviewing this intense martial arts movi...,1
18571,While I have never been a fan of the original ...,-1
13372,This movie is BAD! It's basically an overdone ...,-1


In [44]:
test_df.sample(10)

,text,sentiment
20378,and I still don't know where the hell this mov...,-1
9756,If you are looking for a modern film version o...,1
17608,this is an example of a movie that can have gr...,-1
3402,This movie is certainly one of the greatest fi...,1
21590,What a travesty of movie ratings injustice - a...,-1
18771,Absolutely horrible movie. Not a bad plot conc...,-1
7891,This is an account of events that have been co...,1
12238,Its a feel-good movie that made me feel good. ...,1
7580,I wish I was first exposed to this in a movie ...,1
3593,I'm sure the film contains certain gaps in log...,1


In [76]:
train_df = train_df.sample(int(len(train_df)*0.1))
test_df = test_df.sample(int(len(test_df)*0.1))

In [77]:
train_sentences = train_df.text.values
train_labels = train_df.sentiment.values
test_sentences = test_df.text.values
test_labels = test_df.sentiment.values

In [78]:
import numpy as np
os.makedirs("./datasets/train", exist_ok=True)
np.save("./datasets/train/train_sentences.npy", train_sentences)
np.save("./datasets/train/train_labels.npy", train_labels)
os.makedirs("./datasets/test", exist_ok=True)
np.save("./datasets/test/test_sentences.npy", test_sentences)
np.save("./datasets/test/test_labels.npy", test_labels)


In [79]:
BUCKET_NAME = sess.default_bucket()
PREFIX = 'bert-classification-janossch'

traindata_s3_prefix = f"{PREFIX}/datasets/train"
testdata_s3_prefix = f"{PREFIX}/datasets/test"
output_s3 = f"s3://{BUCKET_NAME}/{PREFIX}/models/"

In [80]:
train_s3 = sess.upload_data(path="./datasets/train/", bucket=BUCKET_NAME, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path="./datasets/test/", bucket=BUCKET_NAME, key_prefix=testdata_s3_prefix)

In [51]:
# TODO: remove - just exploration

import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer
torch.set_grad_enabled(False)

In [52]:
# TODO: remove - just exploration

# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [54]:
# TODO: remove - just exploration

tokens = tokenizer.tokenize(train_sentences[0])
print("Tokens: {}".format(tokens))

Tokens: ['i', 'really', 'liked', 'this', 'summers', '##lam', 'due', 'to', 'the', 'look', 'of', 'the', 'arena', ',', 'the', 'curtains', 'and', 'just', 'the', 'look', 'overall', 'was', 'interesting', 'to', 'me', 'for', 'some', 'reason', '.', 'anyway', '##s', ',', 'this', 'could', 'have', 'been', 'one', 'of', 'the', 'best', 'summers', '##lam', "'", 's', 'ever', 'if', 'the', 'wwf', 'didn', "'", 't', 'have', 'lex', 'lu', '##ger', 'in', 'the', 'main', 'event', 'against', 'yoko', '##zu', '##na', ',', 'now', 'for', 'it', "'", 's', 'time', 'it', 'was', 'ok', 'to', 'have', 'a', 'huge', 'fat', 'man', 'vs', 'a', 'strong', 'man', 'but', 'i', "'", 'm', 'glad', 'times', 'have', 'changed', '.', 'it', 'was', 'a', 'terrible', 'main', 'event', 'just', 'like', 'every', 'match', 'lu', '##ger', 'is', 'in', 'is', 'terrible', '.', 'other', 'matches', 'on', 'the', 'card', 'were', 'razor', 'ramon', 'vs', 'ted', 'di', '##bia', '##se', ',', 'steiner', 'brothers', 'vs', 'heavenly', 'bodies', ',', 'shawn', 'michael

In [55]:
# TODO: remove - just exploration

from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [56]:
# TODO: remove - just exploration

print('Original: ',train_sentences[0])
print('Tokenized: ',tokenizer.tokenize(texts[0]))
print('Token IDs: ',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[0])))

Original:  I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna d

### Tokenize Dataset (moved to the training script)

In [64]:
# TODO: remove - just exploration
input_ids = []
attention_masks = []

for t in train_sentences:
    encoded_dict = tokenizer.encode_plus(
                        t,
                        add_special_tokens = True,
                        max_length = 50,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    
input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

print('Original: ', train_sentences[0])
print('Token IDs: ', input_ids[0])

Original:  I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn Michaels vs Curt Hening, this was the event where Shawn named his big monster of a body guard Diesel, IRS vs 1-2-3 Kid, Bret Hart first takes on Doink then takes on Jerry Lawler and stuff with the Harts and Lawler was always very interesting, then Ludvig Borga destroyed Marty Jannetty, Undertaker took on Giant Gonzalez in another terrible match, The Smoking Gunns and Tatanka took on Bam Bam Bigelow and the Headshrinkers, and Yokozuna d

In [14]:
# TODO: remove - just exploration
from transformers import BertModel

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
input_ids = input_ids.to('cuda')
labels = labels.to('cuda')
model.to('cuda')


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [81]:
train_channel = f"s3://{BUCKET_NAME}/{PREFIX}/datasets/train/"
test_channel = f"s3://{BUCKET_NAME}/{PREFIX}/datasets/test/"

In [87]:
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator
estimator = PyTorchEstimator(
    entry_point="janossch-train.py",
    source_dir="src",
    
    base_job_name="bert-classification",
    output_path=f"s3://{BUCKET_NAME}/{PREFIX}/",
    
    framework_version="1.4.0",
    py_version="py3",
    
    role=role,
    train_instance_count=1,
    train_instance_type="local",
    train_max_run=60*60,
    train_max_wait=60*60,
    
    hyperparameters={
        "seed": 4711,
        "log_level": "DEBUG"
    }
)

In [97]:
estimator.fit({ "train": train_channel, "test": test_channel })

Creating tmpm4epgmtm_algo-1-zqffi_1 ... 
Attaching to tmpm4epgmtm_algo-1-zqffi_12mdone
algo-1-zqffi_1  | 2020-05-05 08:22:35,224 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-zqffi_1  | 2020-05-05 08:22:35,227 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-zqffi_1  | 2020-05-05 08:22:35,240 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-zqffi_1  | 2020-05-05 08:22:35,243 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-zqffi_1  | 2020-05-05 08:22:35,369 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
algo-1-zqffi_1  | Generating setup.py
algo-1-zqffi_1  | 2020-05-05 08:22:35,369 sagemaker-containers INFO     Generating setup.cfg
algo-1-zqffi_1  | 2020-05-05 08:22:35,369 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-zqffi_1  | 2020-05-05 08:22:35,369 sagemaker-conta

algo-1-zqffi_1  | No GPU available, using the CPU instead.
algo-1-zqffi_1  | 2020-05-05 08:24:30,477 sagemaker-containers ERROR    ExecuteUserScriptError:
algo-1-zqffi_1  | Command "/opt/conda/bin/python janossch-train.py --log_level DEBUG --seed 4711"
algo-1-zqffi_1  | INFO:train:Starting!
algo-1-zqffi_1  | DEBUG:train:Namespace(base_job_name='bert-classification-2020-05-05-08-22-32-706', base_model_name='bert-base-uncased', batch_size=64, epochs=4, learning_rate=0.05, log_level='DEBUG', model_dir='/opt/ml/model', num_gpus=0, num_labels=2, output_data_dir='/opt/ml/output/data', output_mode='classification', seed=4711, test='/opt/ml/input/data/test', train='/opt/ml/input/data/train', use_cuda=False)
algo-1-zqffi_1  | DEBUG:train:training function
algo-1-zqffi_1  | DEBUG:train:setting seed: Namespace(base_job_name='bert-classification-2020-05-05-08-22-32-706', base_model_name='bert-base-uncased', batch_size=64, epochs=4, learning_rate=0.05, log_level='DEBUG', model_dir='/opt/ml/model', 

tmpm4epgmtm_algo-1-zqffi_1 exited with code 1
Aborting on container exit...


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpm4epgmtm/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1